# [Module 2.1] 파이프라인으로 생성한 엔드포인트 추론

## 이 노트북은 아래와 같은 작업을 합니다.

- 1. 환경 셋업
- 2. 테스트 데이터 세트 로딩
- 3. 엔드 포인트 추론
- 4. 마지막 신규 엔드포인트 리소스 제거

---    
    

# 1. 환경 셋업

## 기본 세팅 
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

이전 노트북에서 생성한 엔드포인트 이름을 가져 옵니다.


In [ ]:
%store -r depolyment_endpoint_name

In [ ]:
print("depolyment_endpoint_name: \n ", depolyment_endpoint_name)


In [ ]:
import sagemaker
import boto3

role = sagemaker.get_execution_role()
runtime_client = boto3.Session().client('sagemaker-runtime')



# 2. 테스트 데이터 세트 로딩
- 로컬에서 저장된 데이터를 가져와서 데이터를 변환 합니다.
- batch_size 만큼 데이터를 로딩하는 데이터 로더를 정의 합니다.

In [ ]:
import data_utils 
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all(test_num=100)

##  추론을 위한  데이터 세트 로딩
- 전부 데이터를 로딩할 필요가 없지만, 여기서는 기존에 사용한 함수를 이용하기 위해서 전체 데이터를 로드 합니다. 
    - 실제 데이터로 구현시에는 따로이 로드 함수를 사용하시기를 권장 합니다.


In [ ]:
class Params:
    def __init__(self):
        # self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        # self.lr = 0.001
        self.top_k = 10
        self.out = True
        # self.gpu = "0"
                        
args = Params()
print("# of batch_size: ", args.batch_size)


import torch.utils.data as data

test_dataset = data_utils.NCFData(
		test_data, item_num, train_mat, 0, False)

test_loader = data.DataLoader(test_dataset,
		batch_size=args.test_num_ng+1, shuffle=False, num_workers=0)



## 추론할 Paylaod 하나를 생성

In [ ]:
for user, item, label in test_loader:   
    user_np = user.detach().cpu().numpy()
    item_np = item.detach().cpu().numpy()            
    break
payload = {'user':user_np.tolist(), 'item':item_np.tolist()}
print("payload: ", payload)


# 3. 엔드 포인트 추론
- Boto3 invoke_endpoint() 로 추론

### [중요] JSON type 의 payload 를 String 으로 직렬화 해서 제공 함.
```python
payload_dump = json.dumps(payload)
```

In [ ]:
import json, time
from inference_utils import invoke_endpoint
payload_dump = json.dumps(payload)

start_time = time.time()
result = invoke_endpoint(runtime_client, depolyment_endpoint_name, 
                         payload_dump,
                         content_type='application/json'
                        )

print("--- %s seconds ---" % (time.time() - start_time))
print('result: ', result)

# 4. 엔드포인트 리소스 제거


In [ ]:
import boto3

sm_client = boto3.client('sagemaker')
from inference_utils import delete_endpoint
delete_endpoint(client = sm_client, 
                endpoint_name = depolyment_endpoint_name
               )

